In [5]:
import pandas as pd 
import numpy as np
import xml.etree.ElementTree as ET

periode=5 #-------------------------------

tree = ET.parse(f'XMLs\\periode_{periode-1}.xml')
root = tree.getroot()

base_df=pd.read_csv('Data_pl\\base_table_01.csv',index_col='Unnamed: 0')
base_df['Lieferfrist']=base_df['Lieferfrist'].str.replace(',','.').astype('float')
base_df['Abweichung']=base_df['Abweichung'].str.replace(',','.').astype('float')

dfl=[]
for a in root.find('warehousestock').findall('article'):
    dfl.append(a.attrib)
bestands_df = pd.DataFrame(dfl) 

for col in bestands_df.columns:
    try:
        bestands_df[col]=bestands_df[col].astype(int)
    except:
        bestands_df[col]=bestands_df[col].str.replace(',','.')
        bestands_df[col]=bestands_df[col].astype(float)

dfl=[]
for a in root.find('futureinwardstockmovement').findall('order'):
    dfl.append(a.attrib)
futur_orders_df = pd.DataFrame(dfl)

for col in futur_orders_df.columns:
    futur_orders_df[col]=futur_orders_df[col].astype(int)

dfl=[]
for a in root.find('inwardstockmovement').findall('order'):
    dfl.append(a.attrib)
orders_in_df = pd.DataFrame(dfl)

for col in orders_in_df.columns:
    try:
        orders_in_df[col]=orders_in_df[col].astype(int)
    except:
        orders_in_df[col]=orders_in_df[col].str.replace(',','.')
        orders_in_df[col]=orders_in_df[col].astype(float)

In [6]:
periodes=list(range(periode,periode+4))

plan=[#----------------------------------------
['P1',200,200,250,250],
['P2',150,150,150,150],
['P3',100,100,100,100]]

plan=pd.DataFrame(plan,columns=['Pr']+periodes)
plan.index=plan['Pr']
plan.drop(columns=['Pr'],inplace=True)
plan

,5,6,7,8
Pr,,,,
P1,200,200,250,250
P2,150,150,150,150
P3,100,100,100,100


In [7]:
merge_1=pd.merge(base_df,bestands_df,left_on='NR. K',right_on='id').drop(columns=['id','startamount','pct','price','stockvalue'])

for col in periodes:
    merge_1[f'Bedarf_{col}']=merge_1[['P1','P2','P3']]@plan[col]

In [8]:
merge_2=pd.merge(futur_orders_df,base_df,left_on='article',right_on='NR. K')
merge_2['avg_order_time']=merge_2['Lieferfrist']+merge_2['Abweichung']
merge_2=merge_2.drop(columns=['id','NR. K','P1','P2','P3','Diskontmenge','Lieferfrist','Abweichung'])

merge_2['avg_order_time']=merge_2.apply(lambda x : x['avg_order_time'] if x['mode'] == 5 else x['avg_order_time']/2,axis=1)
merge_2['expected_arival_periode']=merge_2['orderperiod']+merge_2['avg_order_time']
merge_2['expected_arival_periode']=np.floor(merge_2['expected_arival_periode'])

In [9]:
# current_col=5
def order_sum_at_periode(x):
    global current_col
    return sum(merge_2[(merge_2['article']==x['NR. K'])&(merge_2['expected_arival_periode']==current_col)]['amount'])

In [10]:
amount_df=merge_1.copy()
current_col=periodes[0]

amount_df[(f'paa_{periodes[0]}','start')]=amount_df['amount']
amount_df[(f'paa_{periodes[0]}','bed')]=amount_df[f'Bedarf_{periodes[0]}']
amount_df[(f'paa_{periodes[0]}','o_in')]=amount_df.apply(order_sum_at_periode,axis=1)

current_col=periodes[0]-1
amount_df[(f'paa_{periodes[0]}','o_in')]+=amount_df.apply(order_sum_at_periode,axis=1)

amount_df[(f'paa_{periodes[0]}','end')]=(amount_df['amount']-amount_df[f'Bedarf_{periodes[0]}']).apply(lambda x : 0 if x<=0 else x)
amount_df[(f'paa_{periodes[0]}','end')]+=amount_df[(f'paa_{periodes[0]}','o_in')]


for col in periodes[1:]:
    current_col=col
    amount_df[(f'paa_{col}','start')]=amount_df[(f'paa_{col-1}','end')]
    amount_df[(f'paa_{col}','bed')]=amount_df[f'Bedarf_{col}']
    amount_df[(f'paa_{col}','o_in')]=amount_df.apply(order_sum_at_periode,axis=1) 
    amount_df[(f'paa_{col}','end')]=(amount_df[(f'paa_{col-1}','end')]-amount_df[f'Bedarf_{col}']).apply(lambda x : 0 if x<=0 else x)
    amount_df[(f'paa_{col}','end')]+=amount_df[(f'paa_{col}','o_in')]


In [12]:
l=[]
for i,x in enumerate(amount_df.columns):
    if type(x)!=tuple:
        l.append((x,''))
    else:
        l.append(x)

amount_df.columns=l
amount_df.columns=pd.MultiIndex.from_tuples(amount_df.columns)

In [13]:
out=amount_df[[f'paa_{x}' for x in periodes]]
out.set_index(amount_df['NR. K']).to_csv(f'Data_pl\\amount_periode_{periode}.csv')

In [14]:
def needed_order_period(x):
    for i,y in enumerate(x):
        if y ==0:
            periode=iter_out.columns[i][0].split('_')[1]
            return int(periode)
    return max(periodes)+1
    

In [15]:
l=[]
for x in out.columns:
    if (x[1]=='o_in')|(x[1]=='start'):
        l.append(x)
iter_out=out.drop(columns=l)

# needed_order_at_start_of_period
amount_df['need_order']=iter_out.apply(needed_order_period,axis=1)+1
# needed_to_order_at_start_of_period
amount_df['need_to_order']=np.floor(amount_df['need_order']-amount_df['Lieferfrist']-amount_df['Abweichung'])
amount_df['need_to_order_iff']=np.floor(amount_df['need_order']-(amount_df['Lieferfrist']+amount_df['Abweichung'])/2)

In [16]:
def min_order_amount(x):
    need_order=int( x['need_to_order'][0] )
    
    if need_order==periode:
        return 'n', x[f'Bedarf_{periode}'][0]
    
    elif need_order<periode:
        return 'f' , x[f'Bedarf_{periode}'][0]
    
    else:
        return None , 0 

In [17]:
amount_df['order_type'],amount_df['min_order_amount']=zip( *amount_df.apply(min_order_amount,axis=1) )

amount_df.to_csv(f'Order_history\\order_amount_{periode}.csv')

In [18]:
amount_df.drop(columns=[
            #(                'NR. K',      ''),
            (                'P1',      ''),
            (                'P2',      ''),
            (                'P3',      ''),
            #(      'Diskontmenge',      ''),
            (            'amount',      ''),
            (          f'Bedarf_{periode}',      ''),
            (          f'Bedarf_{periode+1}',      ''),
            (          f'Bedarf_{periode+2}',      ''),
            (          f'Bedarf_{periode+3}',      ''),])

NR. K Lieferfrist Abweichung Diskontmenge  paa_5                     paa_6  \
                                              start    bed o_in    end  start   
0     21         1.8        0.4          300    450    200    0    250    250   
1     22         1.7        0.4          300    300    150    0    150    150   
2     23         1.2        0.2          300    350    100    0    250    250   
3     24         3.2        0.3         6100   5200   3150    0   2050   2050   
4     25         0.9        0.2         3600   3200   1800    0   1400   1400   
5     27         0.9        0.2         1800   1600    900    0    700    700   
6     28         1.7        0.4         4500   4200   2150    0   2050   2050   
7     32         2.1        0.5         2700   2400   1350    0   1050   1050   
8     33         1.9        0.5          900    400    200    0    200    200   
9     34         1.6        0.3        22000   4000   7200    0      0      0   
10    35         2.2        0.4         3600   3200   1800    0   1400   1400   
11    36         1.2        0.1          900    800    450    0    350    350   
12    37         1.5        0.3          900    800    450    0    350    350   
13    38         1.7        0.4          300    650    450    0    200    200   
14    39         1.5        0.3         1800   2500    900    0   1600   1600   
15    40         1.7        0.2          900    800    450    0    350    350   
16    41         0.9        0.2          900    800    450    0    350    350   
17    42         1.2        0.3         1800   1600    900    0    700    700   
18    43         2.0        0.5         2700    900    450    0    450    450   
19    44         1.0        0.2          900   1950   1350    0    600    600   
20    45         1.7        0.3          900    800    450    0    350    350   
21    46         0.9        0.3          900    800    450    0    350    350   
22    47         1.1        0.1          900    800    450    0    350    350   
23    48         1.0        0.2         1800   1600    900    0    700    700   
24    52         1.6        0.4          600    900    400    0    500    500   
25    53         1.6        0.2        22000  33600  14400    0  19200  19200   
26    57         1.7        0.3          600    600    300    0    300    300   
27    58         1.6        0.5        22000  22400  11000    0  11400  11400   
28    59         0.7        0.2         1800   1600    900    0    700    700   

           ... paa_7 paa_8                 need_order need_to_order  \
      bed  ...   end start    bed o_in end                            
0     200  ...     0     0    250    0   0          8           5.0   
1     150  ...     0     0    150    0   0          7           4.0   
2     100  ...    50    50    100    0   0          9           7.0   
3    3150  ...  2600  2600   3500    0   0          9           5.0   
4    1800  ...     0     0   2000    0   0          7           5.0   
5     900  ...     0     0   1000    0   0          7           5.0   
6    2150  ...     0     0   2350    0   0          7           4.0   
7    1350  ...     0     0   1500    0   0          7           4.0   
8     200  ...     0     0    200    0   0          7           4.0   
9    7200  ...     0     0   7200    0   0          6           4.0   
10   1800  ...     0     0   2000    0   0          7           4.0   
11    450  ...     0     0    500    0   0          7           5.0   
12    450  ...     0     0    500    0   0          7           5.0   
13    450  ...     0     0    500    0   0          7           4.0   
14    900  ...     0     0   1000    0   0          8           6.0   
15    450  ...     0     0    500    0   0          7           5.0   
16    450  ...     0     0    500    0   0          7           5.0   
17    900  ...     0     0   1000    0   0          7           5.0   
18    450  ...     0     0    500    0   0          7           4.0   
